# Project 2

To simplify the problem, I will first break it into two parts: satellite selection and bike routing. 

## Satellite selection

As all satellites are relatively close to the other customers, distance between them and the other customers will not be considered in the satellite selection model. This is similar to saying that the satellite selection will be independent of the bike routes that result from it.

Instead, we will select the satellites based on the requirement that one van per satellite (via back-and-forth trips) can supply the total average demand within 90 minutes. As the satellites have limited capacity, this naturally assumes that bikes are removing the stock at the satellites before the van comes back with more.

To model this problem, we use variables:

$x_s$: binary variable indicating whether satellite $s$ is chosen.
$y_s$: integer variable indicating number of round trips to satellite $s$.

$SA$: set of possible satellites, such that $s \in SA$

The model will minimize cost of renting the satellite locations and of the transport during the 90-minute window.

$ z = \sum\limits_{s \in SA}a_s x_s + b_s y_s$

Where:
* $a_s$ is the daily satellite cost: $a_s = {60, 120, 130, 50}$,
* $b_s$ is the cost of a single trip: $b_s = 2\cdot0.8d_{s} + 100 + 25$
    * where $d_{s}$ is the van-distance from the depot to satellite $s$
    * 100 and 25 are costs associated with each trip given in the problem statement
    
If we let $c_s$ be the capacity at each satellite, we can write the constraints as:

$\sum\limits_{s\in SA}c_s y_s \geq q_{total}$

Where $q_{total}$ is the total average demand at each customer.
This constraint ensures that all trips to the satellites can at least supply the average demand.

$\sum\limits_{s\in SA}x_s \leq SA_{max}$

Where $SA_{max}$ is the maximum number of satellites that may be chosen
This ensures that no more than the maximum number of satellites are chosen.

$y_s \leq x_s \lfloor\frac{g}{2\tau_s}\rceil$

Where $g$ is the target time frame within which we want to deliver all demand to the satellites (here, 90 min), $/tau_s$ is the van-time from depot to satellite $s$. As $g$ is an approximation anyway, we round the number of possible trips to nearest integer.


In [3]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Approximate number of minutes all satellites should be able to receive 
# average total demand from the depot.
load_cycle = 90

distance_matrix = np.genfromtxt('distance_matrix.csv', delimiter=',')

time_matrix = np.genfromtxt('time_matrix.csv', delimiter=',')

# Map satellite indicies to distance_matric indicies
satellite_map = {0: 1,
                 1: 3,
                 2: 7,
                 3: 12}

# Distance from depot for satellite candidates
sat_distance = [distance_matrix[-1][i] for i in satellite_map.values()]

# Time from depot to satellite candidates
sat_time = [time_matrix[-1][i] for i in satellite_map.values()]

# Round trip cost per van trip
trip_cost = [2*.8*sat_distance[i]+100+25 for i in range(len(sat_distance))]

# Daily satellite cost
sat_cost = [60, 120, 130, 50]

# Satellite capacity
cap = [120, 150, 180, 100]

# (Approximate) number of possible trips per load_cycle per satellite
max_trips = [round(load_cycle/(sat_time[i]*2)) for i in range(len(sat_time))]

num_satellites = 4
satellites = range(num_satellites)

model = gp.Model("Satellites") # Make Gurobi model

# ---- Initialize variables

# Whether or not a node is chosen as a satellite
x = model.addVars(num_satellites, vtype=GRB.BINARY, name='x')

# Number of trips to each satellite
y = model.addVars(num_satellites, vtype=GRB.INTEGER, name='y')

# ---- Add constraints

# Ensure that all trips to satellites can supply the average demand
model.addConstr(gp.quicksum(y[i]*cap[i] for i in satellites) >= 390)

# Max satellites is 3
model.addConstr(gp.quicksum(x[i] for i in satellites) <= 3)

# Trips from depot to satellite can only occur if satellite is chosen,
# And number of trips to each depot cannot exceed maximum
model.addConstrs(y[i] <= x[i]*max_trips[i] for i in satellites)


# ---- Set objective

obj = gp.LinExpr()

obj.addTerms(sat_cost, [x[i] for i in satellites])
obj.addTerms(trip_cost, [y[i] for i in satellites])

model.setObjective(obj, GRB.MINIMIZE)

# ---- Optimize model
model.optimize()

# Extract variable values

x_values = []
y_values = []

for i in satellites:
    x_values.append(x[i].X)
    y_values.append(y[i].X)
    
print('\nSatellites: ', x_values)
print('Trips: ', y_values)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1270P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 8 columns and 16 nonzeros
Model fingerprint: 0xf6c5414a
Variable types: 0 continuous, 8 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 4e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 5 rows, 7 columns, 14 nonzeros
Variable types: 0 continuous, 7 integer (4 binary)
Found heuristic solution: objective 595.3760000
Found heuristic solution: objective 587.9664000

Root relaxation: objective 4.663373e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

This solutions suggests we use satellites 2 and 3.

Note that different time frames of 60 and 120 minutes were also tried. With a 60 minute window for deliveries, objective is €642, using satellites 0, 2, and 3 With a 120 minute window for deliveries, objective is €576, and only satellite 3 is used. With a 90 minute window, objective is €582, and satellites 2 and 3 are used. I chose to go with the 90 minute window, as this simulation is based on only averages with no variability, and 2 satellites offer more flexibility. Furthermore, this only costs €6 more.

## Bike tours
The following model is adapted from the paper:

Bard, Jonathan & Huang, Liu & Dror, Moshe & Jaillet, Patrick. (1998). 
A branch and cut algorithm for the VRP with satellite facilities. 
IIE Transactions. 30. 821-834. 10.1023/A:1007500200749. 

The key here is that dummy nodes are made for each satellite to allow a bike to pass through this location multiple times. The formulation is as follows:

Key constants:
* $n$: number of customers, excluding satellites
* $\bar{n}$: number of total nodes
* $m$: number of bikes available
* $Q$: bike capacity
* $\hat{Q}$: maximum inventory on bike before a refill is permitted
* $T$: maximum time per route
* $q_j$: demand at customer $j$
* $p_j$: load/ refill time at customer/satellite $j$
* $\tau_{i,j}$: time to get from customer $i$ to customer $j$
* $T_{i,j}$: 
* $t_{lb}$:
* $\hat{Q}$


The following sets are used:
* $S$: nodes corresponding to a satellite
* $N_{\alpha}$: nodes where bikes can reload (copies of satellites)
* $N$: all nodes
* $I$: all customer nodes, where deliveries must occur
* $I_0$: customer nodes plus satellites (does not include copies)
* $I_F$: customer nodes plus copies of satellites (does not include actual satellites)

The following variables will be optimized:
* $x_{i,j}$: binary variable indicating whether a bike travels from node $i$ to node $j$
* $t_j$: continuous variable indicating time of arrival at node $j$
* $y_j$: continuous variable indicating the load on a bike before node $j$

As we only pay per bike, and not per time or distance travelled, the model will be run with increasing values of $m$ until a feasible solution is reached. Nevertheless, we want to minimize the time travelled on the bikes, so our objective function is:

$min \sum\limits_{i \in N, j \in N}\tau_{ij}x_{ij}$

Subject to the following constraints:

$\sum\limits_{j \in N}x_{ij} = 1 \quad \forall i \in I \qquad (1)$

$\sum_{j \in I}x_{ij} \leq 1 \quad \forall i \in N_{\alpha} \cup S \qquad ()$

$\sum\limits_{i \in N}x_{ji} - \sum\limits_{i \in N}x_{ij} = 0 \quad \forall j \in N \qquad ()$

$\sum\limits_{i \in S}\sum\limits_{j \in I}x_{ij} \leq m \qquad ()$

$t_j \geq t_i + \tau_{ij}x_{ij} + p_j -T_{ij}(1-x_{ij}) \quad \forall i \in I_F \forall j \in N \qquad ()$

$t_j \geq t_{lb} \quad \forall j \in S \qquad ()$

$t_j \geq \min\limits_{i \in S}\tau_{ij} \quad \forall j \in I \qquad ()$

$t_j \leq T-(p_j+\min\limits_{i \in S}\tau_{ij}) \quad \forall j \in \qquad ()$

$t_j \geq \min\limits_{i \in I, s \in S}(\tau_{si}+p_i + \tau_{ij}) \quad \forall j \in N_{\alpha}\qquad ()$

$t_j \leq \max\limits_{i \in I, s \in S}(T-p_j-\tau_{ji}-p_i-\tau_{is}) \quad \forall j \in N_{\alpha}\qquad ()$

$y_j \leq y_i - \bar{q}_{i}x_{ij} + \bar{Q}_i(1-x_{ij}) \quad \forall i \in I_F, \forall j \in N \qquad ()$

$y_j \geq q_j \quad \forall j \in I \qquad ()$

$y_j \leq \hat{Q} \quad \forall j \in N_{\alpha} \qquad ()$


These constraints do the following:
1. Ensures each customer has exactly 1 successor.
. Ensures each (copy of a) satellite has at most one successor.
. Ensures number of arrivals at a node equals the number of departures.
. No more than $m$ vehicles used (specifically, only m vehicles can start from a depot).
. Tracks the time a service begins at node $j$. Also functions to eliminate subtours. 
. Lower bound for arrival back at depot. Must be at least as long as visiting the farthest customer, dropping off the load, and returning to depot.
. Lower bound for $t$ at customer nodes. There must be enough time to get to the customer from the depot.
. Upper bound for $t$ at customer nodes. There must be enough time left for the bike to get back to the depot
. Minimum amount of time to go from depot to customer, provide service, and go to a satellite.
. Latest time to depart satellite/depot $j$, visit a customer, and return to the depot arriving no later than $T$.
. Tracks the load on a bike just prior to visiting node $j$
. Load must be high enough to service the customer
. Load must be under $\hat{Q}$ if bike will stop at a depot/satellite

